<a href="https://colab.research.google.com/github/qweliant/DS-Unit-2-Regression-Classification/blob/master/module3/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# New Section

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
df.SALE_DATE = pd.to_datetime(df.SALE_DATE)
df2 = df

In [6]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,2019-01-01
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,2019-01-01
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,2019-01-01
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,2019-01-01
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,2019-01-01


In [7]:
df.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,0.0,23039.000000,23039.000000,23039.000000,23039.000000,2.303900e+04,23005.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,NaN,10791.186987,1.827423,0.286514,2.320891,3.508555e+03,1819.671724,1.624479,1.328133e+06
std,3713.269887,628.025255,NaN,1103.414210,10.090432,5.778224,12.070554,2.218610e+04,488.376571,0.803898,1.025394e+07
min,1.000000,1.000000,NaN,0.000000,0.000000,-148.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,NaN,10306.000000,0.000000,0.000000,1.000000,4.940000e+02,1920.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,NaN,11211.000000,1.000000,0.000000,1.000000,1.356000e+03,1940.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,NaN,11360.000000,2.000000,0.000000,2.000000,2.280000e+03,1965.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,NaN,11697.000000,750.000000,570.000000,755.000000,1.303935e+06,2019.000000,4.000000,8.500000e+08


In [8]:
df.shape

(23040, 21)

In [0]:
one_family_dwelling = (df.BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS')


In [10]:
df2 = df[one_family_dwelling]
cut = (df2.SALE_PRICE > 100000) & (df2.SALE_PRICE > 2000000)
print(df2.shape)
df2.head()

(5061, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,2019-01-01
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,2019-01-01
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01


In [11]:
df2 = df2[cut] 
print(df2.shape)
df2.head()

(78, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
191,5,OTHER,01 ONE FAMILY DWELLINGS,1,894,155,NaN,A3,5 CROMWELL CIRCLE,NaN,10304.0,1.0,0.0,1.0,"9,927",5048.0,2004.0,1,A3,2425000,2019-01-02
317,3,OTHER,01 ONE FAMILY DWELLINGS,1,7105,58,NaN,A7,412 AVENUE T,NaN,11223.0,1.0,0.0,1.0,"6,000",7320.0,1999.0,1,A7,9200000,2019-01-03
469,1,OTHER,01 ONE FAMILY DWELLINGS,1,1917,121,NaN,A9,123 W 132 STREET,NaN,10027.0,1.0,0.0,1.0,"1,699",3400.0,1910.0,1,A9,2230000,2019-01-04
597,3,OTHER,01 ONE FAMILY DWELLINGS,1,988,56,NaN,A4,393 6TH STREET,NaN,11215.0,1.0,0.0,1.0,"1,800",4017.0,1901.0,1,A4,2902000,2019-01-04
1094,3,OTHER,01 ONE FAMILY DWELLINGS,1,8742,45,NaN,A1,253 DOVER STREET,NaN,11235.0,1.0,0.0,1.0,"5,000",3130.0,1930.0,1,A1,2137500,2019-01-08


In [12]:
df2.LAND_SQUARE_FEET = [ float(i.replace(',','')) for i in df2.LAND_SQUARE_FEET]
df2.dtypes


BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                float64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
APARTMENT_NUMBER                          object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                         float64
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE           

In [13]:
df2.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
191,5,OTHER,01 ONE FAMILY DWELLINGS,1,894,155,NaN,A3,5 CROMWELL CIRCLE,NaN,10304.0,1.0,0.0,1.0,9927.0,5048.0,2004.0,1,A3,2425000,2019-01-02
317,3,OTHER,01 ONE FAMILY DWELLINGS,1,7105,58,NaN,A7,412 AVENUE T,NaN,11223.0,1.0,0.0,1.0,6000.0,7320.0,1999.0,1,A7,9200000,2019-01-03
469,1,OTHER,01 ONE FAMILY DWELLINGS,1,1917,121,NaN,A9,123 W 132 STREET,NaN,10027.0,1.0,0.0,1.0,1699.0,3400.0,1910.0,1,A9,2230000,2019-01-04
597,3,OTHER,01 ONE FAMILY DWELLINGS,1,988,56,NaN,A4,393 6TH STREET,NaN,11215.0,1.0,0.0,1.0,1800.0,4017.0,1901.0,1,A4,2902000,2019-01-04
1094,3,OTHER,01 ONE FAMILY DWELLINGS,1,8742,45,NaN,A1,253 DOVER STREET,NaN,11235.0,1.0,0.0,1.0,5000.0,3130.0,1930.0,1,A1,2137500,2019-01-08


In [0]:
# drop becasue it goes hand in hand with BUILDING_CLASS_CATEGORY, EASE-MENT becasue its mostly NaNs
df2.drop(columns=['TAX_CLASS_AT_PRESENT', 'EASE-MENT', 'APARTMENT_NUMBER'], inplace=True)


In [27]:
df2.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
191,5,0,01 ONE FAMILY DWELLINGS,894,155,A3,5 CROMWELL CIRCLE,10304.0,1.0,0.0,1.0,9927.0,5048.0,2004.0,1,A3,2425000,2019-01-02
317,3,0,01 ONE FAMILY DWELLINGS,7105,58,A7,412 AVENUE T,11223.0,1.0,0.0,1.0,6000.0,7320.0,1999.0,1,A7,9200000,2019-01-03
469,1,0,01 ONE FAMILY DWELLINGS,1917,121,A9,123 W 132 STREET,10027.0,1.0,0.0,1.0,1699.0,3400.0,1910.0,1,A9,2230000,2019-01-04
597,3,0,01 ONE FAMILY DWELLINGS,988,56,A4,393 6TH STREET,11215.0,1.0,0.0,1.0,1800.0,4017.0,1901.0,1,A4,2902000,2019-01-04
1094,3,0,01 ONE FAMILY DWELLINGS,8742,45,A1,253 DOVER STREET,11235.0,1.0,0.0,1.0,5000.0,3130.0,1930.0,1,A1,2137500,2019-01-08


In [31]:
print(df2.BOROUGH.unique(),'\n')
print(df2.NEIGHBORHOOD.unique(), '\n')

print(df2.BUILDING_CLASS_AT_PRESENT.unique(), '\n')
print(df2.BUILDING_CLASS_AT_TIME_OF_SALE.unique(), '\n')
print(df2.ZIP_CODE.unique() , '\n')

['5' '3' '1' '2' '4'] 

[0 1 2 4 'BEDFORD STUYVESANT'] 

['A3' 'A7' 'A9' 'A4' 'A1' 'S1' 'S0' 'A5'] 

['A3' 'A7' 'A9' 'A4' 'A1' 'S1' 'S0' 'A5'] 

[10304. 11223. 10027. 11215. 11235. 11217. 10065. 10014. 10458. 11230.
 11238. 10016. 11231. 11234. 10021. 10011. 10009. 10022. 11357. 11375.
 11226. 11432. 11360. 10024. 10463. 11210. 10028. 10038. 11216. 11201.
 11228. 11205. 11218. 10031. 11694. 11214. 11423. 11222. 10012. 10075.
 10471.] 



In [32]:
df2.replace({'A3':1, 'A7':2, 'A9':3, 'A4':4, 'A1':5, 'S1':6, 'S0':7, 'A5':8})

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
191,5,0,01 ONE FAMILY DWELLINGS,894,155,1,5 CROMWELL CIRCLE,10304.0,1.0,0.0,1.0,9927.0,5048.0,2004.0,1,1,2425000,2019-01-02
317,3,0,01 ONE FAMILY DWELLINGS,7105,58,2,412 AVENUE T,11223.0,1.0,0.0,1.0,6000.0,7320.0,1999.0,1,2,9200000,2019-01-03
469,1,0,01 ONE FAMILY DWELLINGS,1917,121,3,123 W 132 STREET,10027.0,1.0,0.0,1.0,1699.0,3400.0,1910.0,1,3,2230000,2019-01-04
597,3,0,01 ONE FAMILY DWELLINGS,988,56,4,393 6TH STREET,11215.0,1.0,0.0,1.0,1800.0,4017.0,1901.0,1,4,2902000,2019-01-04
1094,3,0,01 ONE FAMILY DWELLINGS,8742,45,5,253 DOVER STREET,11235.0,1.0,0.0,1.0,5000.0,3130.0,1930.0,1,5,2137500,2019-01-08
1387,3,0,01 ONE FAMILY DWELLINGS,1094,72,4,513 11TH STREET,11215.0,1.0,0.0,1.0,1667.0,1824.0,1901.0,1,4,2050000,2019-01-09
1919,3,0,01 ONE FAMILY DWELLINGS,196,6,3,139 BOND STREET,11217.0,1.0,0.0,1.0,563.0,560.0,2000.0,1,3,2300000,2019-01-11
2417,1,1,01 ONE FAMILY DWELLINGS,1398,5,4,101 EAST 63RD STREET,10065.0,1.0,0.0,1.0,2510.0,7349.0,1970.0,1,4,18000000,2019-01-15
2962,1,0,01 ONE FAMILY DWELLINGS,625,14,4,613 HUDSON STREET,10014.0,1.0,0.0,1.0,600.0,2080.0,1899.0,1,4,5100000,2019-01-17
3013,2,0,01 ONE FAMILY DWELLINGS,3309,88,5,265 EAST 203 STREET,10458.0,1.0,0.0,1.0,3152.0,1890.0,1920.0,1,5,5000000,2019-01-17


In [20]:
df2.NEIGHBORHOOD.value_counts()

OTHER                      64
UPPER EAST SIDE (59-79)     7
FOREST HILLS                4
UPPER EAST SIDE (79-96)     2
BEDFORD STUYVESANT          1
Name: NEIGHBORHOOD, dtype: int64

In [22]:
df2.NEIGHBORHOOD.replace({'OTHER': 1, 'UPPER EAST SIDE (59-79)': 2, 'FOREST HILLS':3,  'UPPER EAST SIDE (79-96)':4,  'UPPER EAST SIDE (79-96)':5 , 'BEDFORD STUYVESANT':6}, inplace=True)
df2.NEIGHBORHOOD.value_counts()

0                     64
1                      7
2                      4
4                      2
BEDFORD STUYVESANT     1
Name: NEIGHBORHOOD, dtype: int64

In [24]:
df2.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
191,5,0,01 ONE FAMILY DWELLINGS,1,894,155,NaN,A3,5 CROMWELL CIRCLE,NaN,10304.0,1.0,0.0,1.0,9927.0,5048.0,2004.0,1,A3,2425000,2019-01-02
317,3,0,01 ONE FAMILY DWELLINGS,1,7105,58,NaN,A7,412 AVENUE T,NaN,11223.0,1.0,0.0,1.0,6000.0,7320.0,1999.0,1,A7,9200000,2019-01-03
469,1,0,01 ONE FAMILY DWELLINGS,1,1917,121,NaN,A9,123 W 132 STREET,NaN,10027.0,1.0,0.0,1.0,1699.0,3400.0,1910.0,1,A9,2230000,2019-01-04


In [23]:
df2.APARTMENT_NUMBER.value_counts()

8    1
Name: APARTMENT_NUMBER, dtype: int64

In [0]:
# # train, test = model_selection.train_test_split(cleaned)
# train = df2[(df2['created'].dt.month==4) | (df2['created'].dt.month==5)]
# test = df2[df2['created'].dt.month==6]